# <center>  Python for biologists
    
## Homework 10, Pandas



## Правила

- **Не создавайте доп. веток в репозитории, не копируйте ноутбук.**
- Пишите код только там, где это от вас требуется (`# YOUR CODE HERE`)
- Написав, удаляйте `raise NotImplementedError() `
- Не переименовывайте переменные, если они уже созданы. Происходят автопроверки.
- Под заданием для вас приведены проверки с помощью `assert`. Они падают с ошибкой если решено не правильно и молчат если всё правильно.
- Перед отправкой проверьте что ваш ноутбук работает (юпитер-ноутбук). Нажмите на панели сверху `Kernel` -> `Restart Kernel and Run All Cells...`. Проверье что нет ошибок.
    

#### Погнали!

---


In [1]:
import hashlib
from IPython.display import display

import numpy as np
import pandas as pd


pd.DataFrame.__hash__ = lambda self: int(
    hashlib.sha256(pd.util.hash_pandas_object(self, index=True).values).hexdigest(),
    base=16,
)
pd.core.indexes.base.Index.__hash__ = lambda self: int(
    hashlib.sha256(pd.util.hash_pandas_object(self, index=True).values).hexdigest(),
    base=16,
)
pd.Series.__hash__ = lambda self: int(
    hashlib.sha256(pd.util.hash_pandas_object(self, index=True).values).hexdigest(),
    base=16,
)

    
## I. Обзор дата-фреймов
    
Здесь будем работать с дата-фреймом `olympics`



<a href='https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results'><img src="https://i.ytimg.com/vi/b4wlPz9p4e8/maxresdefault.jpg" align="center" width="500" ></a>

In [2]:
olympics = pd.read_csv('https://github.com/cleuer/olympics/raw/master/src/main/resources/olympic_history_120_years.csv')

#### **Задание 1**💅

Выведите первые 3 строки дата-фрейма

In [3]:
result = olympics.head(3)
display(result)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [4]:
assert len(result) == 3
assert isinstance(result, pd.core.frame.DataFrame)
assert hash(result) == 507558853293347520

#### **Задание 2** 💅

Выведите названия всех колонок в дата-фрейме

In [5]:
result = olympics.columns
display(result)

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [6]:
assert len(result) == 15
assert isinstance(result, pd.core.indexes.base.Index)
assert hash(result) == 204161812668953242

#### **Задание 3** 💅

Сколько колонок имеют `ei` в имени?

In [7]:
result = olympics.columns.str.contains('ei').sum()

In [8]:
assert result == 2

#### **Задание 4** 💅

Выведите размерность дата-фрейма

In [9]:
result = olympics.shape

In [10]:
assert result == (271116, 15)

#### **Задание 5** 💅

Выведите количество NA в колонке `Weight`

In [12]:
result = olympics['Weight'].isna().sum()

62875


In [13]:
assert result == 62875

#### **Задание 6** 💅
 
Выведите количество уникальных значений в колонке `Weight` (для этого есть отдельный метод)

In [14]:
result = olympics['Weight'].nunique()

In [15]:
assert result == 220

#### **Задание 7** 💅

Выведите самое часто-встречаемое значение в колонке `Weight`

In [16]:
result = olympics['Weight'].mode()[0]

In [17]:
assert result == 70.0

#### **Задание 8** 💅

Выведите максимальное значение в колонке `Weight`

In [18]:
result = olympics['Weight'].max()

In [19]:
assert result == 214.0

#### **Задание 9** 💅

Сколько колонок в дата-фрейме имеют тип данных `float64`?

In [20]:
result = len(olympics.select_dtypes(include='float64').sum())

In [21]:
assert result == 3

## II. Отбор данных

#### **Задание 10** 💅

Как зовут атлета в `140228`й по счету строке дата-фрейма?

In [22]:
result = olympics.loc[140227, 'Name']

In [23]:
assert result == 'Yuliya Vyacheslavovna Lipnitskaya'

#### **Задание 11** 💅

В каком городе проходили игры, запись с которых сделана в строке с индексом 360?

In [24]:
result = olympics.loc[360, 'City']

In [25]:
assert result == 'Seoul'

#### **Задание 12** 💅

Отберите все записи относящиеся к мужчинам

In [26]:
result = olympics[olympics['Sex']== 'M']

In [27]:
assert len(result) == 196594
assert hash(result) == 53408147431013694

#### **Задание 13** 💅

Отберите все записи относящиеся летним играм в Барселоне

In [28]:
result = olympics.query('City == "Barcelona"')

In [29]:
assert len(result) == 12977
assert hash(result) == 1257297863566133861

#### **Задание 14** 💅

Какую медаль получила Юлия Липницкая в зимних играх 2014 года в индивидуальном зачёте?

In [30]:
result = olympics.loc[140226, 'Medal']

In [31]:
assert result is np.nan 

#### **Задание 15** 💅

Замените все NA в колонке `Medal` на `"No medal"` (для этого есть специальный метод)

In [32]:
olympics.Medal = olympics.Medal.fillna('No medal')

In [33]:
assert olympics.Medal.isna().sum() == 0
assert olympics.Medal.value_counts().to_dict()['No medal'] == 231333
assert hash(olympics) == 1196581486896556999

#### **Задание 16** 💅

Создайте в дата-фрейме колонку `BMI` с соотвествующими значениями

In [34]:
olympics['BMI'] = olympics.Weight / ((olympics.Height/100) ** 2)

22.784519726470883

In [35]:
assert np.isclose(olympics.BMI.mean(), 22.7845, atol=1e-3)
assert np.isclose(olympics.BMI.std(), 2.9120, atol=1e-3)
assert olympics.BMI.isna().sum() == 64263
assert hash(olympics) == 1059869394349329045

### **Задание XXX**

Нажмите на панели сверху `Kernel` -> `Restart Kernel and Run All Cells...`. Проверьте что нет ошибок.

---
# EDA

В 2 части этого ДЗ вам предстоит написать ваш модуль для EDA. Покажите тут, что у вас получилось!

> По идее, вам нужно только запустить ячейку и всё. Но можете изменить датасет если хотите.

In [36]:
# Можете вставить любой репрезантивный дата-фрейм на ваше усмотрение :)
data_dir = 'https://raw.githubusercontent.com/Python-BI-2023/course_materials/main/data/'
df = pd.read_csv(data_dir + 'titanic.csv')

In [37]:
from my_awesome_eda import run_eda

run_eda(df)

🖖 Greetings, Earthling!

👨🏻‍💻According to my calculations, in the observed table 
The number of observations/rows is 418 
The number of parameters/columns is 11

🤖Categorical columns:['Pclass', 'Sex', 'Embarked']
🤖Numerical columns:['Age', 'SibSp', 'Parch', 'Fare']
🤖String columns:['PassengerId', 'Name', 'Ticket', 'Cabin']

🧮Counts: Pclass
3    218
1    107
2     93
Name: count, dtype: int64

🧮Frequencies: Pclass
3    0.521531
1    0.255981
2    0.222488
Name: proportion, dtype: float64

🧮Counts: Sex
male      266
female    152
Name: count, dtype: int64

🧮Frequencies: Sex
male      0.636364
female    0.363636
Name: proportion, dtype: float64

🧮Counts: Embarked
S    270
C    102
Q     46
Name: count, dtype: int64

🧮Frequencies: Embarked
S    0.645933
C    0.244019
Q    0.110048
Name: proportion, dtype: float64

🔢Age statistics:
📉Min: 0.17
📈Max: 76.0
😒Mean: 30.272590361445783
✍🏼Std: 14.181209235624422
📊Q0.25: 21.0
⚖️Median: 27.0
📊Q0.75: 39.0


🔢SibSp statistics:
📉Min: 0
📈Max: 8
😒Mean: 0.

---

Надеюсь, у тебя всё получилось! Но в любом случае - ты молодец:)🥳

---